Voordat het programma Theorio vragenbeheer kan runnen, moeten de volgende stappen worden uitgevoerd:

1. Zorg ervoor dat alle libraries zijn geïnstalleerd
# pip install -r requirements.txt

2. Zorg ervoor dat de .env file is aangemaakt
met API_KEY= [API key uit verslag]

3. Run het programma, maar niet cell voor cell want dat vind Tkinter niks.

LET OP: De data komt van de api, en die heeft een cold start. Deze duurt ongeveer 10 seconden. Hierna is alles snel.

In [ ]:
# Standaard Python libraries
import asyncio
import datetime
import io
import json
import os
import threading
import tkinter as tk
from tkinter import messagebox
from tkinter import ttk

# Third-party libraries
import aiohttp
from dotenv import load_dotenv
from PIL import Image, ImageTk

Structuur van de Code

De applicatie is opgebouwd uit drie kerncomponenten:
- NavigatieBalk: Bovenaan bevindt zich een navigatiebalk waarmee je kunt schakelen tussen 'Examens', 'Onderdelen', 'Feedback Formulieren' en 'Exporteren van Rapporten'. Deze is altijd aanwezig.

- LijstFrame: Bij het klikken op een knop in de navigatiebalk wordt het LijstFrame bijgewerkt met een lijst van relevante gegevens voor de gekozen sectie. Hier vind je 
bijvoorbeeld een overzicht van beschikbare examens of ontvangen feedback.

- DetailsFrame: Wanneer je een item uit het LijstFrame selecteert, wordt de inhoud weergegeven in het grotere DetailsFrame. Dit frame past zich dynamisch aan op basis van het type inhoud—of het nu gaat om verschillende vraagtypes, feedbackdetails of rapporten.

Alle Data wordt opgehaald van de Google Cloud API, de functies hiervan vind je in de BACKEND folder.

We beginnen met wat setup code, deze is niet zo interessant.

In [ ]:
# Configuratie voor de API
API_CONFIG = {
    'ENDPOINTS': {
        'subjects': '/http-getAllSubjects',
        'exams': '/http-getAllExams',
        'feedback': '/http-getAllFeedback',
        'update_feedback': '/http-updateFeedbackStatus',
        'create_question': '/http-createQuestion',
        'update_question': '/http-updateQuestion',
        'delete_question': '/http-deleteQuestion'
    }
}

# Vraag types voor de popup menu
QUESTION_TYPES = [
    {
        'id': 'multiple_choice',
        'title': 'Multiple Choice',
        'description': 'Een vraag met meerdere antwoordopties waarvan er één correct is.',
        'icon': '📝'
    },
    {
        'id': 'open',
        'title': 'Open Vraag',
        'description': 'Een vraag waar de student een eigen antwoord moet formuleren.',
        'icon': '️✏️'
    },
    {
        'id': 'image_selection',
        'title': 'Afbeelding Selectie',
        'description': 'Een vraag waarbij de student de juiste afbeelding moet selecteren.',
        'icon': '🖼️'
    },
    {
        'id': 'drag_and_drop',
        'title': 'Drag and Drop',
        'description': 'Een vraag waarbij de student items naar de juiste positie moet slepen.',
        'icon': '🎯'
    }
]

# Helper functies 
def format_date(timestamp):
    """Maak een leesbare datum van een timestamp"""
    if isinstance(timestamp, dict) and '_seconds' in timestamp:
        return datetime.fromtimestamp(timestamp['_seconds']).strftime('%d-%m-%Y %H:%M')
    return ''

De Navigatiebalk roept de functies toon_onderdelen, toon_examens, toon_rapporten en toon_feedback aan in de app class.
Deze functies updaten de UI van het LijstFrame. (Zie de app class)

In [ ]:
class NavigatieBalk(tk.Frame):
    """Navigation bar met knoppen en logo"""
    
    def __init__(self, parent, app):
        super().__init__(parent)
        self.pack(fill='x', padx=10, pady=10)
        self.create_nav_buttons(app)
        self.add_logo()
    
    def create_nav_buttons(self, app):
        """Maak de knoppen voor de navigatie"""
        buttons = [
            # Sommige knoppen hebben een lambda functie om de async functie aan te roepen, deze zijn async omdat ze data ophalen van de API en dan pas de UI updaten
            ("📘 Onderdelen", lambda: app.handle_async_button(app.toon_onderdelen())),
            ("📝 Examens", lambda: app.handle_async_button(app.toon_examens())),
            ("📊 Rapporten", app.toon_rapporten),
            ("💬 Feedback", lambda: app.handle_async_button(app.toon_feedback()))
        ]
        
        for text, command in buttons:
            btn = ttk.Button(self, text=text, command=command)
            btn.pack(side="left", padx=5, pady=10)
    
    def add_logo(self):
        """Aan de rechterkant van de navigatiebalk een logo"""
        try:
            logo = Image.open("logo.png")
            logo = logo.convert("RGBA")
            resized_logo = logo.resize(128, 56) #De logo moet transparent zijn en kleiner
            self.logo = ImageTk.PhotoImage(resized_logo)
            self.logo_label = tk.Label(self, image=self.logo)
            self.logo_label.pack(side="right")
        except Exception as e:
            print("Error bij het laden van het logo: ", e)

De class App is de belangrijkste class in de applicatie, deze bindt alle andere classes aan elkaar en beheert de logica achter de applicatie en houdt de data bij die in de UI wordt getoond. Hier zitten vooral functies in die via andere classes worden aangeroepen.


In [ ]:
class App:
    """Main application class met alle globale functies"""
    
    # Start functies
    def __init__(self):
        self.setup_api_config()
        self.setup_window()
        self.setup_async_loop()
        self.setup_frames()

    def setup_api_config(self):
        """Haal de API key en url op uit de .env file"""
        load_dotenv()
        self.api_url = os.getenv('API_URL')
        self.api_key = os.getenv('API_KEY')
        self.api_client = APIClient(self.api_url, self.api_key)
        if not self.api_key or not self.api_url:
            raise ValueError("API_KEY en/of API_URL niet gevonden in .env file, check het verslag voor de api_key en api_url")

    def setup_window(self):
        """Venster instellingen"""
        self.root = tk.Tk()
        self.root.title("Theorio Cursusbeheer")
        self.root.geometry("1200x800")
        self.root.configure(bg='#3374FF') #blauw

    def setup_frames(self):
        """Frames opzetten"""
        self.nav_frame = NavigatieBalk(self.root, self)
        self.frame = tk.Frame(self.root, bg='#3374FF')
        self.frame.pack(fill='both', expand=True)
        self.lijst_frame = LijstFrame(self.frame, self)
        self.details_frame = DetailsFrame(self.frame, self)

    # Async loop setup, Maak een aparte loop, in een aparte thread, voor async functies, zodat de GUI niet vastloopt 
    def setup_async_loop(self):
        """Maak de async event loop aan"""
        self.loop = asyncio.new_event_loop()
        asyncio.set_event_loop(self.loop)
        self.loop_thread = threading.Thread(target=self._run_event_loop, daemon=True)
        self.loop_thread.start()

    def _run_event_loop(self):
        """Run de async event loop"""
        asyncio.set_event_loop(self.loop)
        self.loop.run_forever()

    def handle_async_button(self, coro):
        """Async knop, voert de coroutine (async functie) in een aparte thread uit zodat de GUI niet vastloopt"""
        asyncio.run_coroutine_threadsafe(coro, self.loop)

    #! Navigatie knoppen - Aangeroepen vanuit NavigatieBalk class
    async def show_onderdelen(self):
        """Onderdelen ophalen van de API en tonen in de lijst"""
        self.lijst_frame.update_lijst([], loading=True) #Loading indicator aanzetten
        try:
            data = await self.api_client.get('subjects')
            items = [self.format_subject_data(subject) for subject in data['subjects']] # Pas de data aan naar de verwachte structuur van de lijst
            self.lijst_frame.update_lijst(items, loading=False) #Loading indicator uitzetten en lijst tonen
        except Exception as e:
            self.show_error(f"Fout b fetching subjects: {str(e)}")
            self.lijst_frame.update_lijst([], loading=False) #Loading indicator uitzetten bij fout en lege lijst tonen

    async def show_exams(self):
        """Examens ophalen van de API en tonen in de lijst"""
        self.lijst_frame.update_lijst([], loading=True) #Loading indicator aanzetten
        try:
            data = await self.api_client.get('exams')

            # De data vanuit de server is eigenlijk 3 arrays voor elk onderdeel in het examen (gevaarherkenning, inzicht, kennis)
            # Maar we maken hier 3 aparte items in de lijst voor elk onderdeel dus je krijgt Examen 1 - Gevaarherkenning, Examen 1 - Inzicht, Examen 1 - Kennis etc.
            # Idealiter zou dit al gedaan moeten zijn op de server side, maar dit werkt ook
            exams = []
            for exam in data['exams']:
                exam_categories = []
                
                categories = {
                    'gevaarherkenning': 'Gevaarherkenning',
                    'inzicht': 'Inzicht',
                    'kennis': 'Kennis'
                }
                
                for category, display_name in categories.items():
                    if category in exam and exam[category].get('questions'):
                        category_questions = exam[category]['questions']
                        exam_categories.append({
                            'titel': f"Examen {exam['id']} - {display_name}", #Examen 1 - Gevaarherkenning, etc.
                            'vragen': [self.format_question_data(i, question) # Pas de data aan naar de verwachte structuur van de lijst, en details frame
                                     for i, question in enumerate(category_questions)]
                        })
                
                exams.extend(exam_categories)
            
            self.lijst_frame.update_lijst(exams, loading=False) #Loading indicator uitzetten en lijst updaten
        except Exception as e:
            self.show_error(f"Error fetching exams: {str(e)}")
            self.lijst_frame.update_lijst([], loading=False)

    def show_rapporten(self):
        """Rapporten ophalen van de API en tonen in de lijst"""
        items = ["Export feedback"] #Alleen export feedback is beschikbaar (geen loading nodig, want het niet uit de API komt), helaas had ik geen tijd meer om de andere rapporten te implementeren
        self.lijst_frame.update_lijst(items, loading=False) #Lijst tonen

    async def show_feedback(self):
        """Feedback ophalen van de API en tonen in de lijst"""
        self.lijst_frame.update_lijst([], loading=True) #Loading indicator aanzetten
        try:
            data = await self.api_client.get('feedback')
            self.lijst_frame.update_lijst([data.get('feedback', [])], loading=False) #Loading indicator uitzetten en lijst tonen
        except Exception as e:
            self.show_error(f"Error fetching feedback: {str(e)}")
            self.lijst_frame.update_lijst([], loading=False)

    #! Update detail frames - Aangeroepen wanneer je een item selecteert in de lijst
    def show_vraag_details(self, hoofdstuk, vraag_data):
        """Toon de vraag details in het details frame"""
        self.details_frame.toon_vraag_ui(hoofdstuk, vraag_data)
    
    def show_feedback_details(self, feedback_data):
        """Toon de feedback details in het details frame"""
        self.details_frame.show_feedback_ui(feedback_data)

    async def show_rapport_feedback_details(self):
        """Toon info van de csv bestand over de feedback in het details frame"""
        try:
            data = await self.api_client.get('feedback')
            if data:
                self.details_frame.show_feedback_rapport_ui(data.get('feedback', []))
        except Exception as e:
            self.show_error(f"Error exporting feedback: {str(e)}")

    #! Data Formatting Methods
    # Idealiter zou dit niet nodig zijn, maar de data die de API terug geeft is niet helemaal in de verwachte structuur en ik heb geen tijd meer om deze aan te passen
    def format_subject_data(self, subject):
        """Format subject data voor de lijst"""
        return {
            'titel': subject['title'],
            'vragen': [self.format_question_data(i, question) 
                      for i, question in enumerate(subject.get('questions', []))]
        }
    
    # Gebruikt in show_onderdelen en show_exams
    def format_question_data(self, index, question):
        """Format question data voor de lijst"""
        return {
            'titel': f'Vraag {index+1}',
            'vraag_tekst': question['question'],
            'type': question['type'],
            'opties': question.get('answers', []) if question.get('type') == 'multiple_choice' else [],
            'antwoord': question.get('correctAnswer', ''),
            'uitleg': question.get('explanation', ''),
            'afbeelding': question.get('image', ''),
            'context': question.get('context', ''),
            'imageOptions': question.get('imageOptions', []),
            'terms': question.get('terms', {}),
            'correctPositions': question.get('correctPositions', []),
            'correctAnswer': question.get('correctAnswer', 0),
            'id': question['id']
        }
    
    #! Error handling
    def show_error(self, message):
        """Foutmelding tonen in een popup"""
        print(f"Error: {message}")
        tk.messagebox.showerror("FOUT:", message)

    #! Applicatie start en cleanup
    def run(self):
        """Run de applicatie"""
        try:
            self.root.mainloop()
        finally:
            self.cleanup()

    def cleanup(self):
        """Cleanup, dit is nodig om de async loop te stoppen en de thread te joinen zodat de app niet crasht bij het sluiten"""
        # Used in: run method
        self.loop.call_soon_threadsafe(self.loop.stop)
        self.loop_thread.join()
        self.loop.close()


De lijstframe wordt geupdate zodra er een knop in de navigatiebalk wordt geklikt. de belangrijkste functie is de update_lijst functie, die een array van items krijgt en deze in de treeview zet.

In [ ]:
class LijstFrame(tk.Frame):
    """De lijstframe is de linker frame die de lijst met onderdelen, examens en feedback toont"""
    #! Setup
    def __init__(self, parent, app):
        super().__init__(parent)
        self.pack(fill='both', side='left', padx=10, pady=10, expand=False)
        self.configure(width=300)
        self.pack_propagate(False) #Zorgt ervoor dat de lijstframe niet groter wordt dan de inhoud
        self.app = app
        self.vraag_data = {} #Dit is een dictionary die de data van de vragen opslaat zodat deze gemakkelijk kunnen worden opgehaald in de on_select functie
        self.feedback_data = {} #Zelfde als hierboven, maar voor de feedback data

        self.container = tk.Frame(self)
        self.container.pack(fill='both', expand=True)

        # Loading label
        self.loading_label = tk.Label(
            self.container,
            text="🔄 Ophalen van data...",
            font=('Arial', 12),
            pady=20
        )

        # Style voor de Treeview
        style = ttk.Style()
        style.configure("Custom.Treeview", rowheight=40)
        style.configure("Custom.Treeview.Item", padding=5)

        # Treeview
        self.tree = ttk.Treeview(self.container, style="Custom.Treeview")
        self.tree.pack(fill='both', expand=True)
        
        # Popup menu, voor het maken van nieuwe vragen
        self.popup_menu = tk.Menu(self, tearoff=0)
        self.popup_menu.add_command(label="+ Maak vraag", command=self.create_new_question)
        
        # Wanneer er op een item in de treeview wordt geklikt, roepen we de on_select functie aan
        self.tree.bind('<<TreeviewSelect>>', self.on_select)

    #! Update functies
    def update_lijst(self, items, loading=True):
        """Update de lijst met items, en toon de loading label als loading=True, check ook wat voor soort items er zijn"""
        if loading:
            self.tree.pack_forget() #Verberg de treeview
            self.loading_label.pack(fill='both', expand=True)
            self.update()
            return

        self.loading_label.pack_forget() #Verberg de loading label
        self.tree.pack(fill='both', expand=True) #Toon de treeview
        
        #Verwijder alle items uit de vraag_data dictionary
        self.vraag_data.clear()
        self.feedback_data.clear()
        for item in self.tree.get_children(): #Verwijder alle items uit de treeview
            self.tree.delete(item)
            
        # Voeg de nieuwe items toe aan de treeview
        for item in items:
            #de meest omslachtige check, maar het werkt en had echt geen tijd meer om er een betere te maken
            if isinstance(item, list) and all(isinstance(x, dict) and 'feedback' in x for x in item): #Feedback lijst
                # This is the feedback list
                for feedback in item:
                    unique_id = f"feedback-{feedback['id']}" #Maak een unieke id voor de feedback, anders vind de treeview deze niet leuk, ook kunnen wea dan kijken bij on_select of het feedback is
                    self.feedback_data[unique_id] = feedback #sla de feedback op in de feedback_data dictionary
                    
                    # Leuke emoji's voor de status
                    status_emoji = {
                        'in_progress': '🔄',
                        'completed': '✅',
                        'pending': '⏳'
                    }.get(feedback.get('status', 'new'), '🆕')
                    
                    # Format de datum
                    date_str = format_date(feedback.get('date', {}))
                    
                    # Maak de display text
                    display_text = f"{status_emoji} {date_str} - {feedback.get('subject', 'Geen onderwerp')}"
                    
                    self.tree.insert("", "end", iid=unique_id, text=display_text)
            
            # Onderdelen en examens zijn dictionaries met een titel en een lijst van vragen
            elif isinstance(item, dict): 
                parent = self.tree.insert("", "end", text=item['titel'])
                
                # Voeg de knop "[+ Nieuwe vraag toevoegen]" toe aan het hoofdstuk
                self.tree.insert(parent, "end", 
                               text="+ Nieuwe vraag toevoegen", 
                               tags=('add_button',), #Hierdoor kunnen wij in on_select vinden wat er is geklikt
                               values=(parent,)) #Values zijn voor extra informatie, zodat we later de parent kunnen vinden
                
                # Voeg de vragen toe aan het hoofdstuk
                for vraag in item['vragen']:
                    unique_id = item['titel']+'-'+vraag['id']+'-'+vraag['titel'] #Maak een unieke id voor de vraag, anders vind de treeview deze niet leuk
                    self.vraag_data[unique_id] = vraag #Sla de vraag op in de vraag_data dictionary
                    self.tree.insert(parent, "end", iid=unique_id, text=vraag['vraag_tekst'])
            else: #Rapporten zijn strings (maar dit is ook niet future proof, want ik wil het later meer rapporten toevoegen)
                self.tree.insert("", "end", text=str(item))
            
            #In de toekomst zou ik deze functie willen herschrijven zodat het meer flexibel is, met eventueel een extra parameter voor het type item (onderdeel, examen, feedback, rapport) 🤷‍♂️

    def create_new_question(self):
        """Maak een nieuwe vraag aan"""
        if hasattr(self, 'selected_parent'):
            parent_text = self.tree.item(self.selected_parent)['text']
            
            # Laat de gebruiker een type kiezen in een popup modal
            dialog = QuestionTypeDialog(self)
            self.wait_window(dialog) #Wacht tot de gebruiker een type heeft gekozen
            
            # Als er een type is gekozen
            if dialog.result:
                # Maak een lege vraag template
                empty_question = {
                    'id': 'new',
                    'titel': 'Nieuwe vraag',
                    'vraag_tekst': '',
                    'type': dialog.result,
                    'opties': [],
                    'antwoord': '',
                    'uitleg': '',
                    'afbeelding': '',
                    'context': '',
                    'terms': {},
                    'imageOptions': [],
                    'correctPositions': [],
                    'parent': parent_text
                }
                
                # Toon de vraag details met de lege template
                self.app.show_vraag_details(parent_text, empty_question)

    def on_select(self, event):
        """Wanneer er op een item in de treeview wordt geklikt, roepen we deze functie aan, er kunnen 4 dingen gebeuren:
        1. Er is op de knop "[+ Nieuwe vraag toevoegen]" geklikt, dan maken we een nieuwe vraag aan
        2. Er is op een vraag geklikt, dan tonen we de vraag details
        3. Er is op een feedback item geklikt, dan tonen we de feedback details
        4. Er is op een rapport geklikt, dan tonen we de rapport details (Voor nu altijd de feedback rapporten)"""
        selection = self.tree.selection()
        if not selection: #Als er niks is geselecteerd, stop
            return
        
        selected_item = selection[0] #Pak het eerste item, want we doen niet aan meerdere selectie
        selected_text = self.tree.item(selected_item)['text']
        
        # Er is op de knop "Export feedback" geklikt, dan tonen we de feedback rapport details
        if selected_text == "Export feedback":
            self.app.handle_async_button(self.app.show_rapport_feedback_details())
            return

        print(f"Selected item: {selected_item}")
        # Er is op een feedback item geklikt, dan tonen we de feedback details
        if selected_item.startswith('feedback-'):
            feedback_data = self.feedback_data.get(selected_item)
            print(f"Feedback data: {feedback_data}")
            if feedback_data:
                self.app.show_feedback_details(feedback_data)
            return
        
        # Er is op de knop "[+ Nieuwe vraag toevoegen]" geklikt, dan maken we een nieuwe vraag aan
        if 'add_button' in self.tree.item(selected_item)['tags']:
            # Get parent from values
            parent_id = self.tree.item(selected_item)['values'][0]
            parent_text = self.tree.item(parent_id)['text']
            self.selected_parent = parent_id
            self.create_new_question()
            return
        
        # Er is op een vraag geklikt, dan tonen we de vraag details
        parent_id = self.tree.parent(selected_item)
        
        if parent_id:  # Dit is een vraag, want er is een parent dus examen of onderdeel, dit is niet future proof want stel we maken later andere items met dropdowns dan moet dit worden aangepast
            vraag_data = self.vraag_data.get(selected_item)
            if vraag_data:
                parent_text = self.tree.item(parent_id)['text']
                self.app.show_vraag_details(
                    hoofdstuk=parent_text,
                    vraag_data=vraag_data
                )


Dit is de popup die komt wanneer je op de knop "[+ Nieuwe vraag toevoegen]" drukt. Deze popup geeft de gebruiker de keuze uit de verschillende vraagtypes. Deze popup leeft in de 'top level', wat betekent dat deze een apart venster opent dat los staat van het hoofdvenster.

In [ ]:
class QuestionTypeDialog(tk.Toplevel):
    """Popup menu voor het kiezen van het type van een vraag"""

    #! Setup
    def __init__(self, parent):
        super().__init__(parent)
        self.result = None
        
        self.title("Kies vraag type")
        self.geometry("400x500")
        self.resizable(False, False)
        
        # Center van het scherm
        self.center_window()

        #Zorgt ervoor dat de popup gekoppeld wordt aan het parent-venster en dat de user interactie met de popup de focus houdt
        self.transient(parent)
        self.grab_set()
        
        self.create_widgets()

    def center_window(self):
        """Center de popup op het scherm"""
        self.update_idletasks()
        width = self.winfo_width()
        height = self.winfo_height()
        x = (self.winfo_screenwidth() // 2) - (width // 2)
        y = (self.winfo_screenheight() // 2) - (height // 2)
        self.geometry(f'+{x}+{y}')

    #! Maak de UI elementen aan
    def create_widgets(self):
        """Maak de UI elementen aan"""
        # Title
        tk.Label(self, text="Kies het type vraag", font=('Arial', 16, 'bold'), pady=20).pack()

        # Container voor de vraag types
        types_frame = tk.Frame(self)
        types_frame.pack(fill='both', expand=True, padx=20, pady=10)

        for type_info in QUESTION_TYPES:
            self.create_type_button(types_frame, type_info) 

    #! Maak een knop aan voor elk type vraag
    def create_type_button(self, parent, type_info):
        """Maak een knop aan voor een vraag type"""
        type_frame = tk.Frame( parent, relief='solid', borderwidth=1,cursor='hand2')
        type_frame.pack(fill='x', pady=5, ipady=10)

        def on_click(event):
            self.select_type(type_info['id'])

        # Icon and title container
        header = tk.Frame(type_frame)
        header.pack(fill='x', padx=10, pady=(5, 0))
        
        # Icon
        icon_label = tk.Label(header, text=type_info['icon'], font=('Arial', 14), cursor='hand2')
        icon_label.pack(side='left')

        # Title
        title_label = tk.Label( header, text=type_info['title'], font=('Arial', 12, 'bold'), cursor='hand2')
        title_label.pack(side='left', padx=10)

        # Description
        desc_label = tk.Label(type_frame, text=type_info['description'], wraplength=350, justify='left', cursor='hand2')
        desc_label.pack(fill='x', padx=10, pady=(5, 0))

        # Alles moet dezelfde events binden, want als je op de tekst klikt, moet het ook werken
        for widget in [header, icon_label, title_label, desc_label, type_frame]:
            widget.bind('<Button-1>', on_click)

    def select_type(self, type_id):
        """Selecteer een vraag type"""
        self.result = type_id
        self.destroy() #Sluit de popup


De Api client is de class die de requests naar de API verwerkt, deze kan CRUD requests versturen naar de API.

In [ ]:
class APIClient:
    """API client class"""
    def __init__(self, base_url, api_key):
        self.base_url = base_url
        self.headers = {
            'x-api-key': api_key,
            'Content-Type': 'application/json'
        }
    
    async def get(self, endpoint_key):
        """GET request"""
        return await self._request(endpoint_key, 'get')
    
    async def post(self, endpoint_key, data):
        """POST request"""
        return await self._request(endpoint_key, 'post', data)
    
    async def put(self, endpoint_key, data):
        """PUT request"""
        return await self._request(endpoint_key, 'put', data)
    
    async def delete(self, endpoint_key, data=None):
        """DELETE request"""
        return await self._request(endpoint_key, 'delete', data)
    
    async def _request(self, endpoint_key, method, data=None):
        """Request handler"""
        async with aiohttp.ClientSession() as session:
            try:
                endpoint = f"{self.base_url}{API_CONFIG['ENDPOINTS'][endpoint_key]}" #Endpoint opbouwen
                async with getattr(session, method)(
                    endpoint,
                    headers=self.headers,
                    json=data
                ) as response:
                    # Zowel 200 als 201 worden als succes status codes geaccepteerd, dit is namelijk niet consistent in de API
                    if response.status in [200, 201]:
                        return await response.json()
                    error_text = await response.text()
                    raise Exception(f"API Fout: {response.status}, {error_text}")
            except Exception as e:
                raise Exception(f"Request gefaald: {str(e)}")


De DetailsFrame is de rechter frame die de details van een vraag, feedback of rapport toont. Dit is echt een draak van een frame, omdat er zoveel verschillende dingen kunnen worden getoond. Bij nader inzien had ik beter een class voor kunnen maken die de verschillende soorten frames kan aanmaken.